In [2]:
import pandas as pd

## Metadata

In [18]:
dfm = pd.read_csv('../data/item_metadata.csv',dtype="str", sep=",", encoding="utf-8")
dfm_stack = dfm.set_index('item_id', drop=False, append=True).properties.str.split('|', expand=True).stack()
dfm.head()

,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


In [19]:
dfm_stack.head()

   item_id   
0  5101     0       Satellite TV
            1        Golf Course
            2    Airport Shuttle
            3    Cosmetic Mirror
            4       Safe (Hotel)
dtype: object

In [ ]:
dfm_sparse = pd.get_dummies(dfm_stack, prefix=None, prefix_sep=None).groupby(level=1, sort=False).agg(max)
dfm_sparse.head()

In [ ]:

for s in dfm_sparse.columns:
    print(s)